In [13]:
def createDataSet():
    dataSet = [[1, 1, 'yes'],
            [1, 1, 'yes'],
            [1, 0, 'no'],
            [0, 1, 'no'],
            [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels


In [14]:
import math


def calcShannonEnt(dataset):
    numEntries = len(dataset)
    labelCounts = {}

    for featVec in dataset:
        currentLabel = featVec[-1]

        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1


    shannonEnt = 0.0
    for key in labelCounts:
        prob = labelCounts[key] / numEntries
        shannonEnt += -1 * prob * math.log2(prob)

    return shannonEnt

calcShannonEnt(createDataSet()[0])

0.9709505944546686

In [15]:
def splitDataSet(dataset, index, value):
    """
    splitDataSet(通过遍历dataSet数据集，求出index对应的colnum列的值为value的行)
        就是依据index列进行分类，如果index列的数据等于 value的时候，就要将 index 划分到我们创建的新的数据集中
    Args:
        dataSet 数据集                 待划分的数据集
        index 表示每一行的index列        划分数据集的特征
        value 表示index列对应的value值   需要返回的特征的值。
    Returns:
        index列为value的数据集【该数据集需要排除index列】
    """
    retDataSet = []
    for featVec in dataset:
        if featVec[index] == value:
            reducedFeatVec = featVec[:index]
            reducedFeatVec.extend(featVec[index+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet
splitDataSet(createDataSet()[0],1,1)

[[1, 'yes'], [1, 'yes'], [0, 'no'], [0, 'no']]

In [16]:
def chooseBestFeatureToSplit(dataset):
    """chooseBestFeatureToSplit(选择最好的特征)

    Args:
        dataSet 数据集
    Returns:
        bestFeature 最优的特征列
    """
    numFeatures = len(dataset[0]) - 1
    baseEntropy = calcShannonEnt(dataset)

    bestInfoGain, bestFeatureIndex = 0.0, -1

    # 遍历每一个特征，计算最优的信息增益的列
    for i in range(numFeatures):
        featList = [example[i] for example in dataset]
        uniqueVals = set(featList)

        newEntropy = 0.0
        for value in uniqueVals:
            subdataset = splitDataSet(dataset, i, value)
            newEntropy += len(subdataset) / len(dataset) * calcShannonEnt(subdataset)

        infoGain = baseEntropy - newEntropy
        print("infoGain=", infoGain, "baseFeature: ", i)
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeatureIndex = i

    return bestFeatureIndex

In [17]:
def majorityCnt(classList):
    """majorityCnt(选择出现次数最多的一个结果)
    Args:
        classList label列的集合
    Returns:
        bestFeature 最优的特征列
    """
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
    if classCount['yes'] > classCount['no']:
        return 'yes'
    else:
        return 'no'
        

In [20]:
def createTree(dataset, labels):
    classList = [example[-1] for example in dataset]

    # 第一个停止条件：所有类标签完全相同
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    
    # 第二个停止条件：使用完了所有的特征，仍然不能将数据集划分成仅有唯一类别的分组
    if len(dataset[0]) == 1:
        return majorityCnt(classList)

    
    bestFeat = chooseBestFeatureToSplit(dataset)
    bestFeatLabel = labels[bestFeat]

    myTree = {bestFeatLabel:{}}
    # 注: labels列表是可变对象，在PYTHON函数中作为参数时传址引用，能够被全局修改
    # 所以这行代码导致函数外的同名变量被删除了元素，造成例句无法执行，提示'no surfacing' is not in list
    del(labels[bestFeat])

    # 取出最优列，然后它的branch做分类
    featValues = [example[bestFeat] for example in dataset]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        # 求出剩余的标签label
        subLabels = labels[:]
        # 遍历当前选择特征包含的所有属性值，在每个数据集划分上递归调用函数createTree()
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataset, bestFeat, value), subLabels)
        print('myTree', value, myTree)
    return myTree

In [21]:
if __name__ == "__main__":
    dataset, labels = createDataSet()
    myTree = createTree(dataset, labels)
    print(myTree)

infoGain= 0.4199730940219749 baseFeature:  0
infoGain= 0.17095059445466854 baseFeature:  1
myTree 0 {'no surfacing': {0: 'no'}}
infoGain= 0.9182958340544896 baseFeature:  0
myTree 0 {'flippers': {0: 'no'}}
myTree 1 {'flippers': {0: 'no', 1: 'yes'}}
myTree 1 {'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
